In [1]:
import pandas as pd

from src.basic_description_utils import describe_data
from src.geo_utils import fill_missing_location_row
from src.date_correction_utils import parse_date, correct_dates
from src.nlp_utils import classify_books
from src.regex_patterns import find_patterns_in_column
from src.customers_utils import detect_gender_row, fill_education_and_occupation

# SentenceTransformer rises a warning during import when used in Jupyter. It doesn't affect the code execution.

C:\Users\topal\Desktop\Billups\codeChallengeProject\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Data Import

In [2]:
input_dataframes = {
    'books': pd.read_csv('../data/books.csv'),
    'checkouts': pd.read_csv('../data/checkouts.csv'),
    'customers': pd.read_csv('../data/customers.csv'),
    'libraries': pd.read_csv('../data/libraries.csv')
}

input_dataframes["libraries"].rename(columns={"region": "state"}, inplace=True)
input_dataframes["customers"].rename(columns={"zipcode": "postal_code"}, inplace=True)

### Basic Raw Data Description

In [3]:
input_data_description, input_data_row_counts = describe_data(df_dict=input_dataframes)

In [4]:
input_data_description

nunique  count_nan  regex_match
          column name                                    
books     id                  240          0          240
          title               220          0          180
          authors             171         67            6
          publisher            64        146           42
          publishedDate       171          2          167
          categories           87         39          200
          price               223          2          212
          pages               203          0          224
checkouts id                  240          0         2000
          patron_id          2000          0         2000
          library_id           18          0         2000
          date_checkout       573         65         1730
          date_returned       622         58         1730
customers id                 2000          0         2000
          name               1996          0         1894
          street_address     1915          0         1853
          city                 32         94         1809
          state                 9        102         1801
          postal_code         194          1            0
          birth_date         1830        107         1707
          gender                8        101         1856
          education            39        104         1810
          occupation           73        107          997
libraries id                   18          0           18
          name                 18          0            9
          street_address       18          0           11
          city                  5          4            5
          state                 4          2           13
          postal_code          14          3            4

In [5]:
input_data_row_counts

books         240
checkouts    2000
customers    2000
libraries      18
Name: row counts, dtype: int64

### Data Correction

In [6]:
corrected_data = {key: pd.DataFrame() for key in input_dataframes.keys()}

#### Books
* **id** - It was in an appropriate format.
* **title** - Basic string processing (removing unnecessary blank spaces) cleared the column.
* **authors** - This column will be excluded from further analysis due to the significant number of missing rows, complicated format and questionable contribution.
* **publisher** - More than 50% of the column is empty. Even if the column were filled, it would still not provide significant information.
* **publishedDate** - As the column elements mostly contain the year only, I used the regex pattern for 4-digit sequence detection.
* **categories** - After visual inspection, I noticed multiple labels for practically the same categories (or related enough) that I could merge. Initially, I used the SBERT model to vectorize categories and basic K-means clustering to merge the labels. After further inspection, I noticed a significant mismatch between initial categories and book titles, so I decided to apply the same model to the titles and create a new classification.
* **price** - Most prices were in the appropriate form (220/240). Some prices contained currency code (USD) or some special character. I applied the regex pattern to extract a 2-decimal float. Additionally, two empty fields were filled with the median price from the same book category. 
* **pages** - Pages were cleared by basic string processing. 

In [7]:
# uniform data
for col in input_dataframes["books"].columns:
    if col not in ["authors", "publisher"]:
        current_col = input_dataframes["books"][col]
        corrected_data["books"][col] = find_patterns_in_column(current_col, "books", col)

corrected_data["books"]["price"] = corrected_data["books"]["price"].astype(float)
corrected_data["books"]["pages"] = corrected_data["books"]["pages"].astype(int)

# new book classification
category_map = {
    0: "medicine",
    1: "financial",
    2: "mechanics", 
    3: "advertising",
    4: "government publication",
    5: "engines"
 }
corrected_data["books"]["categories"] = classify_books(column=corrected_data["books"]["title"], n_clusters=6).replace(category_map)

# fill missing price
median_price_per_category = corrected_data["books"].groupby("categories")["price"].median().round(2)
corrected_data["books"]["price"] = corrected_data["books"].apply(lambda x: x["price"] if x["price"] == x["price"]
else float(median_price_per_category[x["categories"]]), axis=1)

#fill missing publishing date
corrected_data["books"]["publishedDate"] = corrected_data["books"]["publishedDate"].fillna(value=corrected_data["books"]["publishedDate"].mode().iloc[0])

#### Checkouts
* **id/patron_id/library_id** - It was in an appropriate format.
* **date_checkout/date_returned** - The dates were cleared and formatted using basic string preprocessing and the python-dateutil package. I detected unrealistic dates (from the future or distant past) by the regex pattern. I used custom designed logic to recreate wrong years.

In [8]:
# uniform data
for col in input_dataframes["checkouts"].columns:
    current_col = input_dataframes["checkouts"][col]
    if "date" in col:
        current_col = current_col.apply(parse_date)
    corrected_data["checkouts"][col] = find_patterns_in_column(current_col, "checkouts", col)
    
corrected_checkouts, corrected_returns = correct_dates(
    input_dataframe=input_dataframes["checkouts"], 
    corrected_dataframe=corrected_data["checkouts"]
)

corrected_data["checkouts"].loc[corrected_checkouts.index, "date_checkout"] = corrected_checkouts[
    "date_checkout"].apply(lambda x: x.strftime("%Y-%m-%d") if pd.notnull(x) else None)

corrected_data["checkouts"].loc[corrected_returns.index, "date_returned"] = corrected_returns[
    "date_returned"].apply(lambda x: x.strftime("%Y-%m-%d") if pd.notnull(x) else None)

#### Customers
* **id** - It was in an appropriate format.
* **name** - Names were cleared by basic string processing and basic regex pattern.
* **street_address/city/state/zipcode** - After string uniforming, missing fields were filled using the geopy detector. The detector expects a comma-separated string containing known location parameters and returns a full location description (if possible). Missing addresses were replaced by the most frequent addresses within the same postal code.
* **birth_date** - Same as in checkouts, after cleaning the data there were some unrealistic timestamps. In my opinion, there is no point in generating artificial dates of birth. If I decide to use the birthdate to calculate the customer's age, I will probably fill in the missing age by the most frequent.
* **gender** - After the label uniforming (all existing strings modified to match one of the two labels "male" and "female"), the missing fields were filled using the gender-guesser package. The model takes a name as an input and provides a gender label as an output. Names whose gender the model couldn't recognise were labelled "unknown."
* **education/occupation** - After string uniforming, missing fields were filled with the most frequent label from the appropriate group. For example, if the occupation had been missing, it would be filled with the most frequent occupations of the same education and gender. In case of both fields were missed, they were filled with the most frequent values in the whole set. 

In [9]:
# uniform data
for col in input_dataframes["customers"].columns:
    current_col = input_dataframes["customers"][col]
    if "date" in col:
        current_col = current_col.apply(parse_date)
    corrected_data["customers"][col] = find_patterns_in_column(current_col, "customers", col)

# fill location using geopy
corrected_data["customers"] = corrected_data["customers"].apply(lambda row: fill_missing_location_row(row), axis="columns")
# fill missing streets with most frequent street per postal code
mode = lambda x: x.mode() if type(x.mode) == str else x.mode()[0]
most_frequent_street_per_postal_code = corrected_data["customers"].groupby("postal_code")["street_address"].agg(mode)
corrected_data["customers"]["street_address"] = corrected_data["customers"].apply(lambda row: row["street_address"] 
if row["street_address"] is not None else most_frequent_street_per_postal_code[row["postal_code"]], axis=1)

corrected_data["customers"] = corrected_data["customers"].apply(lambda row: fill_missing_location_row(row), axis="columns")

# fill rest of the missing data by manually found data
corrected_data["customers"].loc[corrected_data["customers"]["city"].str.contains("302"), "city"] = "Troutdale"
corrected_data["customers"]["state"] = corrected_data["customers"]["state"].fillna('oregon')

# fill gender
corrected_data["customers"] = corrected_data["customers"].apply(lambda row: detect_gender_row(row=row), axis="columns")

# fill occupation and education
occupation_groups = corrected_data["customers"].groupby(["occupation", "gender"])["education"]
education_groups = corrected_data["customers"].groupby(["education", "gender"])["occupation"]
corrected_data["customers"] = corrected_data["customers"].apply(
    lambda row: fill_education_and_occupation(
        row=row,
        education_groups=education_groups,
        occupation_groups=occupation_groups
    ),
    axis="columns"
)

# fill rest of the education and occupation
most_frequent_occupation = corrected_data["customers"]["occupation"].mode().iloc[0]
corrected_data["customers"]["occupation"] = corrected_data["customers"]["occupation"].fillna(most_frequent_occupation)

most_frequent_education = corrected_data["customers"]["education"].mode().iloc[0]
corrected_data["customers"]["education"] = corrected_data["customers"]["education"].fillna(most_frequent_education)

#### Libraries
*  **id** - Already was in good format.
* **name** - Names were cleared by basic string processing and basic regex pattern.
* **street_address/city/state/zipcode** - Same as it was done for the customers.

In [10]:
# uniform data
for col in input_dataframes["libraries"].columns:
    corrected_data["libraries"][col] = find_patterns_in_column(input_dataframes["libraries"][col], "libraries", col)
# Fill missing data
corrected_data["libraries"] = corrected_data["libraries"].apply(lambda row: fill_missing_location_row(row), axis="columns")
corrected_data["libraries"]["state"] = corrected_data["libraries"]["state"].replace({"or": "oregon"})

### Counts After Correction 

In [11]:
corrected_data_description, corrected_data_row_counts = describe_data(df_dict=corrected_data, skip=["authors", "publisher"])

In [12]:
corrected_data_description

nunique  count_nan  regex_match
          column name                                    
books     id                  240          0          240
          title               212          0          240
          publishedDate       113          0          240
          categories            6          0          240
          price               222          0          240
          pages               196          0          240
checkouts id                  240          0         2000
          patron_id          2000          0         2000
          library_id           18          0         2000
          date_checkout       454         65         1935
          date_returned       468         58         1942
customers id                 2000          0         2000
          name               1995          0         2000
          street_address     1842          0         2000
          city                 19          0         2000
          state                 2          0         2000
          postal_code          64          0         2000
          birth_date         1644        293         1707
          gender                3          0         2000
          education             4          0         2000
          occupation            7          0         2000
libraries id                   18          0           18
          name                 17          0           18
          street_address       18          0           18
          city                  1          0           18
          state                 1          0           18
          postal_code          13          0           18

### Save Preprocessed Data

In [13]:
for key in corrected_data.keys():
    corrected_data[key].to_csv(f"../data/preprocessed_{key}.csv", index=False)